<a href="https://colab.research.google.com/github/MdelaVilla/MORS/blob/main/Sesion_transformers_MORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial extraido y traducido de
https://huggingface.co/docs/transformers/tasks/sequence_classification

En este tutorial vamos a usar Transformers para una tarea de Clasificación de textos, en este caso de análisis de sentimiento para determinar la etiqueta (positivo, negativo o neutral) a asignar a una secuencia de texto.

Veremos:
1. Como hacer un finetune (ajuste fino) a un modelo preentrenado (DistilBert) con un dataset (IMDB) para determinar si una 'review' de una película es positiva o negativa.
2. Usaremos nuestro modelo ajustado para inferir nuevas predicciones.


Antes de empezar, instalemos las librerias necesarias

In [1]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


Es recomendable usar una cuenta de Hugging Face para subir y compartir nuestro modelo con la comunidad.

Si no la tienes, create una cuenta y crea un token (de escritura). Introducelo para loguearte.

In [2]:
from huggingface_hub import notebook_login

In [22]:
notebook_login()


Comencemos cargando el dataset IMDb desde la librería Datasets de HF

In [4]:
from datasets import load_dataset

imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Veamos un ejemplo del contenido del dataset IMDb.
Hay dos campos en el dataset:
text: el texto de la revisión que hace el usuario
label: el valor con que se etiqueta, 0 para revisión negativa, 1 para positiva

In [5]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

# Preproceso

Vamos a empezar cargando el tokenizador del modelo preentrenado 'Distilbert' para preprocesar el campo 'text'

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Creamos una función de preprocesamiento para tokenizar texto y truncar secuencias para que no superen la longitud máxima de entrada de DistilBERT:

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

Para aplicar la función de preprocesamiento a todo el conjunto de datos, usamos la función 'map' de la librería Datasets. Puede acelerar el mapeo estableciendo *batched=True* para procesar varios elementos del dataset a la vez:

In [8]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Ahora creamos un lote de ejemplos usando DataCollatorWithPadding. Es más eficiente rellenar dinámicamente, uno a uno, los textos hasta la longitud más larga de un lote durante la clasificación, que rellenar de una vez todo el conjunto de datos hasta la longitud máxima.
Estamos usando la librería TensorFlow.

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

# Evaluación

Incluir una métrica durante el entrenamiento suele ser útil para evaluar el rendimiento del modelo. Vamos a cargar rápidamente un método de evaluación desde la librería *Evaluate*. Para esta tarea, cargamos la métrica *accuracy*:

In [10]:
import evaluate

accuracy = evaluate.load("accuracy")

Después creamos una función que pase sus predicciones y etiquetas para calcular la precisión:

In [11]:
import numpy as np


def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

Nuestra función *compute_metrics* está lista para funcionar. Volveremos a ella cuando configuremos el entrenamiento.

# Entrenamiento

Antes de empezar a entrenar el modelo, creamos un mapa que nos convierta los identificadores esperados y sus etiquetas con id2label y label2id:

In [12]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

label2id = {"NEGATIVE": 0, "POSITIVE": 1}

Para ajustar un modelo en TensorFlow, comience configurando una función optimizadora, un programa (schedule) para la tasa de aprendizaje y algunos hiperparámetros de entrenamiento:

In [13]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_imdb["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

Ahora podemos cargar 'DistilBERT' con TFAutoModelForSequenceClassification junto con el número de etiquetas esperadas y las conversiones de etiquetas:

In [14]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Convertimos nuestros datasets (IMDb) al formato tf.data.Dataset con prepare_tf_dataset():

In [15]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_imdb["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_imdb["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Configuramos el modelo para entrenar con el método *compile*. Tenga en cuenta que todos los modelos de Transformers tienen una función de pérdida relevante predeterminada para la tarea, por lo que no necesita especificar una a menos queramos.

In [16]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

Las dos últimas cosas que debe configurar antes de comenzar el entrenamiento son calcular la precisión (*accuracy*) de las predicciones y proporcionar una manera de enviar el modelo al Hub. Ambos se realizan mediante llamadas de la librería Keras.

Pasamos nuestra función *compute_metrics* a *KerasMetricCallback*:


In [17]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

E indicamos dónde queremos enviar el modelo y el tokenizador obtenidos en *PushToHubCallback*:

In [23]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="mi_prezioso_modelo",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/mdelavilla/mi_prezioso_modelo into local empty directory.


Luego, agrupe sus devoluciones de llamada (*callbacks*):

In [24]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3
  63/1562 [>.............................] - ETA: 27:56:59 - loss: 0.5042

Una vez que se completa el entrenamiento (puede durar... mucho), el modelo se carga automáticamente en el Hub.

# Predicción

Ahora que hemos *ajustado* un modelo, podemos usarlo para realizar inferencias o predicciones. Indica algún texto sobre el que te gustaría realizar inferencias:

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

La forma más sencilla de probar el modelo ajustado para predicción es usarlo en una *pipeline()*. Creamos una instancia de *pipeline* para el análisis de sentimiento con nuestro modelo y le pasamos el texto:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="mdelavilla/mi_prezioso_modelo")
classifier(text)

O sin el pipeline, paso a paso:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mdelavilla/mi_prezioso_modelo")
inputs = tokenizer(text, return_tensors="tf")

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("mdelavilla/mi_prezioso_modelo")
logits = model(**inputs).logits

In [ ]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]